Тренировочный корпус, состоящий из 114,911 положительных, 111,923 отрицательных записей с платформы микроблогинга Twitter.
http://study.mokoron.com/

– id: уникальный номер сообщения в системе twitter;

– tdate: дата публикации сообщения (твита);

– tname: имя пользователя, опубликовавшего сообщение;

– ttext:  текст сообщения (твита);

– ttype: поле в котором в дальнейшем будет указано к кому классу относится твит (положительный, отрицательный, нейтральный);

– trep: количество реплаев к данному сообщению. В настоящий момент API твиттера не отдает эту информацию;

– trtv: ???

– tfav: число сколько раз данное сообщение было добавлено в избранное другими пользователями;

– tstcount: число всех сообщений пользователя в сети twitter;

– tfol: количество фоловеров пользователя (тех людей, которые читают пользователя);

– tfrien: количество друзей пользователя (те люди, которых читает пользователь);

– listcount: количество листов-подписок в которые добавлен твиттер-пользователь.

Рубцова Ю. Автоматическое построение и анализ корпуса коротких текстов (постов микроблогов) для задачи разработки и тренировки тонового классификатора //Инженерия знаний и технологии семантического веба. – 2012. – Т. 1. – С. 109-116.

In [1]:
import pandas as pd

names = ['id', 'tdate', 'tname', 'ttext', 'ttype', 'trep', 'trtv',
         'tfav','tstcount', 'tfol', 'tfrien', 'listcount']
positive = pd.read_csv("../data/positive.csv", sep=';', names=names)
negative = pd.read_csv("../data/negative.csv", sep=';', names=names)
data = pd.concat([positive[:110000], negative[:110000]])
names = list(filter(lambda x: x not in ['ttext', 'ttype'], names))
data = data.drop(columns=names)
data.loc[data['ttype'] == -1, 'ttype'] = 0
data = data.sample(frac=1).reset_index(drop=True)

In [2]:
data.head(5), data.shape

(                                               ttext  ttype
 0  RT @Jmurkaa: Короч ребят фото гавно,и у тетки ...      1
 1  пт 13\nи контрольный тест\nсилы зла, я на ваше...      1
 2  Не люблю я геометрию:( еще а зачету готовиться...      0
 3  @JulJulianapai  Здравствуйте, Юлиана! Смейтесь...      1
 4  @safirova а сами то , микс из какашек и говняш...      1, (220000, 2))

In [3]:
X, y = list(data['ttext']), list(data['ttype'])

In [4]:
from pymystem3 import Mystem

lemmatizer = Mystem()
X = list(map(lemmatizer.lemmatize, X))

In [5]:
from nltk.corpus import stopwords
from string import punctuation

with open('../data/mystopwords') as f:
    stop_words = [w.replace('\n', '') for w in f]

tmp_func = lambda tokens: [token for token in tokens if token not in stop_words and len(token) > 3]

X = list(map(tmp_func, X))

In [6]:
import re

def filtring_data(list_string):
    pattern = u"[а-яА-ЯёЁ]+"
    tmp_list = list(filter(lambda x: re.fullmatch(pattern, x), list_string))
    regex = re.compile(r'^.*(.)(\1)(\1).*$')
    return list(filter(lambda x: not re.fullmatch(regex, x), tmp_list))
X = list(map(filtring_data, X))

In [7]:
print(X[:5])

[['короча', 'ребята', 'фото', 'гавно', 'тетка', 'жопа', 'гавно', 'просто', 'видно'], ['контрольный', 'тест', 'сила', 'сторона'], ['любить', 'геометрия', 'зачет', 'готовиться'], ['здравствовать', 'юлиана', 'смеяться', 'пять', 'минута', 'смех', 'прибавлять', 'жизнь', 'студент', 'равноценный', 'стакан', 'сметана'], ['микс', 'какашка', 'говняшка']]


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range=(1, 2), min_df=15, max_df=0.0006, max_features=10000)
#vectorizer = CountVectorizer(ngram_range=(1, 2), min_df=10, max_df=0.0006, max_features=10000)
bag_of_words = vectorizer.fit_transform(list(map(" ".join, X)))

In [9]:
vectorizer.get_feature_names()[:30]

['аарон',
 'ааха',
 'аахах',
 'аахахах',
 'ааххах',
 'абзац',
 'абонент',
 'аваад',
 'авария',
 'аватар',
 'аватарка',
 'авахгуй',
 'авахгуй баярлуулна',
 'август',
 'австрия',
 'авто',
 'автобус ехать',
 'автограф',
 'автоматически',
 'автомобиль',
 'автор',
 'авторитет',
 'авторский',
 'автошкола',
 'агент',
 'агентство',
 'агрессивный',
 'агрессия',
 'агутин',
 'адам']

In [13]:
from sklearn.decomposition import TruncatedSVD

In [22]:
from sklearn.decomposition import TruncatedSVD

tSVD = TruncatedSVD(n_components=3)
tSVD.fit(bag_of_words.T)

TruncatedSVD(algorithm='randomized', n_components=3, n_iter=5,
             random_state=None, tol=0.0)

In [34]:
n_train = 180000
X_train, X_test = tSVD.components_.T[:n_train], tSVD.components_.T[n_train:]
#X_train, X_test = bag_of_words.toarray()[:n_train], bag_of_words.toarray()[n_train:]
y_train, y_test = y[:n_train], y[n_train:]

In [35]:
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
fitted_gnb = gnb.fit(X_train, y_train)

report = classification_report(y_test, fitted_gnb.predict(X_test), target_names=['Negative', 'Positive'])
print(report)

              precision    recall  f1-score   support

    Negative       0.50      0.89      0.64     19922
    Positive       0.52      0.12      0.19     20078

    accuracy                           0.50     40000
   macro avg       0.51      0.50      0.42     40000
weighted avg       0.51      0.50      0.42     40000



In [36]:
from sklearn.neighbors import KNeighborsClassifier

knc = KNeighborsClassifier()
fitted_knc = knc.fit(X_train, y_train)

report = classification_report(y_test, fitted_knc.predict(X_test), target_names=['Negative', 'Positive'])
print(report)

              precision    recall  f1-score   support

    Negative       0.55      0.68      0.61     19922
    Positive       0.59      0.45      0.51     20078

    accuracy                           0.56     40000
   macro avg       0.57      0.57      0.56     40000
weighted avg       0.57      0.56      0.56     40000



In [37]:
from sklearn.neighbors import KNeighborsClassifier

knc = KNeighborsClassifier(n_neighbors=20, n_jobs=-1)
fitted_knc = knc.fit(X_train, y_train)

report = classification_report(y_test, fitted_knc.predict(X_test), target_names=['Negative', 'Positive'])
print(report)

              precision    recall  f1-score   support

    Negative       0.54      0.71      0.61     19922
    Positive       0.58      0.40      0.48     20078

    accuracy                           0.56     40000
   macro avg       0.56      0.56      0.54     40000
weighted avg       0.56      0.56      0.54     40000



In [38]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=100, max_depth=10, n_jobs=-1)
fitted_rfc = rfc.fit(X_train, y_train)

report = classification_report(y_test, fitted_rfc.predict(X_test), target_names=['Negative', 'Positive'])
print(report)

              precision    recall  f1-score   support

    Negative       0.52      0.62      0.57     19922
    Positive       0.54      0.43      0.48     20078

    accuracy                           0.53     40000
   macro avg       0.53      0.53      0.52     40000
weighted avg       0.53      0.53      0.52     40000



In [ ]:
from sklearn.svm import SVC

svc = SVC()
fitted_svc = svc.fit(X_train, y_train)

report = classification_report(y_test, fitted_svc.predict(X_test), target_names=['Negative', 'Positive'])
print(report)

/home/ficusss_g53/.local/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
